In [1]:
# from doctr.models import detection
import torch
import os
from doctr.models import ocr_predictor
from collections import OrderedDict
from doctr.io import DocumentFile
import cv2

/home/abhayram/Robust-word-detector-for-Indic-Documents/venv/lib/python3.6/site-packages/doctr/local.py:17: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.fernet import Fernet


In [2]:
#Loading the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictor = ocr_predictor(pretrained=True).to(device)

# original saved file with DataParallel
state_dict = torch.load("./weights/db_resnet50.pt")

#COMMENT THE BELOW LINES OF CODE IF DataParallel was not used during training (provided weights used DataParallel)
# create new OrderedDict that does not contain `module.`
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
    
    
predictor.det_predictor.model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [3]:
def doctr_predictions(directory):
    #Gets the predictions from the model
    
    doc = DocumentFile.from_images(directory)
    result = predictor(doc)
    dic = result.export()
    
    page_dims = [page['dimensions'] for page in dic['pages']]
    
    regions = []
    abs_coords = []
    
    regions = [[word for block in page['blocks'] for line in block['lines'] for word in line['words']] for page in dic['pages']]
    abs_coords = [
    [[int(round(word['geometry'][0][0] * dims[1])), 
      int(round(word['geometry'][0][1] * dims[0])), 
      int(round(word['geometry'][1][0] * dims[1])), 
      int(round(word['geometry'][1][1] * dims[0]))] for word in words]
    for words, dims in zip(regions, page_dims)
    ]

#     pred = torch.Tensor(abs_coords[0])
    return abs_coords

In [4]:
#loading the image file and running inference
image_file = "./test.png"
pred = doctr_predictions(image_file)

In [5]:
#creating output image
image = cv2.imread(image_file)
for w in pred[0]:
    cv2.rectangle(image,(w[0], w[1]),(w[2], w[3]),(0,255,0))
cv2.imwrite(f"output.png", image)

True